# Create LLM Agent using OpenVINO

LLM are limited to the knowledge on which they have been trained and the additional knowledge provided as context, as a result, if a useful piece of information is missing the provided knowledge, the model cannot “go around” and try to find it in other sources. This is the reason why we need to introduce the concept of Agents.

The core idea of agents is to use a language model to choose a sequence of actions to take. In agents, a language model is used as a reasoning engine to determine which actions to take and in which order. Agents can be seen as applications powered by LLMs and integrated with a set of tools like search engines, databases, websites, and so on. Within an agent, the LLM is the reasoning engine that, based on the user input, is able to plan and execute a set of actions that are needed to fulfill the request.

![agent](https://github.com/openvinotoolkit/openvino_notebooks/assets/91237924/22fa5396-8381-400f-a78f-97e25d57d807)

[LangChain](https://python.langchain.com/docs/get_started/introduction) is a framework for developing applications powered by language models. LangChain comes with a number of built-in agents that are optimized for different use cases.

This notebook explores how to create an AI Agent step by step using OpenVINO and LangChain.

#### Table of contents:

- [Prerequisites](#Prerequisites)
- [Create tools](#Create-tools)
- [Create prompt template](#Create-prompt-template)
- [Create LLM](#Create-LLM)
  - [Download model](#Select-model)
  - [Select inference device for LLM](#Select-inference-device-for-LLM)
- [Create agent](#Create-agent)
- [Run the agent](#Run-agent)
- [Interactive Demo](#Interactive-Demo)
  - [Use built-in tool](#Use-built-in-tool)
  - [Create customized tools](#Create-customized-tools)
  - [Create AI agent demo with Gradio UI](#Create-AI-agent-demo-with-Gradio-UI)


## Prerequisites

[back to top ⬆️](#Table-of-contents:)


In [ ]:
import os

os.environ["GIT_CLONE_PROTECTION_ACTIVE"] = "false"

%pip install -Uq pip
%pip uninstall -q -y optimum optimum-intel
%pip install --pre -Uq openvino openvino-tokenizers[transformers] --extra-index-url https://storage.openvinotoolkit.org/simple/wheels/nightly
%pip install -q --extra-index-url https://download.pytorch.org/whl/cpu\
"git+https://github.com/huggingface/optimum-intel.git"\
"git+https://github.com/openvinotoolkit/nncf.git"\
"torch>=2.1"\
"datasets"\
"accelerate"\
"gradio"\
"transformers>=4.38.1" "langchain>=0.2.3" "langchain-huggingface>=0.0.3" "wikipedia"

## Create a tools

[back to top ⬆️](#Table-of-contents:)

First, we need to create some tools to call. In this example, we will create 3 custom functions to do basic calculation. For [more information](https://python.langchain.com/docs/modules/tools/) on creating custom tools.


In [ ]:
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int


@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent

Tools are interfaces that an agent, chain, or LLM can use to interact with the world. They combine a few things:

1. The name of the tool
2. A description of what the tool is
3. JSON schema of what the inputs to the tool are
4. The function to call
5. Whether the result of a tool should be returned directly to the user


In [ ]:
print(f"name of `multiply` tool: {multiply.name}")
print(f"description of `multiply` tool: {multiply.description}")

Now that we have created all of them, and we can create a list of tools that we will use downstream.


In [ ]:
tools = [multiply, add, exponentiate]

## Create prompt template

[back to top ⬆️](#Table-of-contents:)

A prompt for a language model is a set of instructions or input provided by a user to guide the model's response, helping it understand the context and generate relevant and coherent language-based output, such as answering questions, completing sentences, or engaging in a conversation.

Different agents have different prompting styles for reasoning. In this example, we will use [ReAct agent](https://react-lm.github.io/) with its typical prompt template. For a full list of built-in agents see [agent types](https://python.langchain.com/docs/modules/agents/agent_types/).

![react](https://github.com/openvinotoolkit/openvino_notebooks/assets/91237924/a83bdf7f-bb9d-4b1f-9a0a-3fe4a76ba1ae)

A ReAct prompt consists of few-shot task-solving trajectories, with human-written text reasoning traces and actions, as well as environment observations in response to actions. ReAct prompting is intuitive and flexible to design, and achieves state-of-the-art few-shot performances across a variety of tasks, from question answering to online shopping!

In an prompt template for agent, `input` is user's query and `agent_scratchpad` should be a sequence of messages that contains the previous agent tool invocations and the corresponding tool outputs.


In [ ]:
PREFIX = """<|system|>\nRespond to the human as helpfully and accurately as possible. You have access to the following tools:"""

FORMAT_INSTRUCTIONS = """Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per $JSON_BLOB, as shown:

```
{{{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}}}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}}}
```"""

SUFFIX = """Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation:.
Thought:\n<|end|>"""

HUMAN_MESSAGE_TEMPLATE = (
    "<|user|>\n{input}\n<|end|>\n\n<|assistant|>\n{agent_scratchpad}"
)

## Create LLM

[back to top ⬆️](#Table-of-contents:)

Large Language Models (LLMs) are a core component of LangChain. LangChain does not serve its own LLMs, but rather provides a standard interface for interacting with many different LLMs. In this example, we select `neural-chat-7b-v3-1` as LLM in agent pipeline.

**phi3-mini-instruct** - The Phi-3-Mini is a 3.8B parameters, lightweight, state-of-the-art open model trained with the Phi-3 datasets that includes both synthetic data and the filtered publicly available websites data with a focus on high-quality and reasoning dense properties. More details about model can be found in [model card](https://huggingface.co/microsoft/Phi-3-mini-4k-instruct), [Microsoft blog](https://aka.ms/phi3blog-april) and [technical report](https://aka.ms/phi3-tech-report).

### Download model

[back to top ⬆️](#Table-of-contents:)

To run LLM locally, we have to download the model in the first step. It is possible to [export your model](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#export) to the OpenVINO IR format with the CLI, and load the model from local folder.


In [ ]:
from pathlib import Path

model_id = "microsoft/Phi-3-mini-4k-instruct"
model_path = "Phi-3-mini-4k-instruct-ov-int4"

if not Path(model_path).exists():
    !optimum-cli export openvino --model {model_id} --task text-generation-with-past --trust-remote-code --weight-format int4 {model_path}

### Select inference device for LLM

[back to top ⬆️](#Table-of-contents:)


In [ ]:
import openvino as ov
import ipywidgets as widgets

core = ov.Core()

support_devices = core.available_devices
if "NPU" in support_devices:
    support_devices.remove("NPU")

device = widgets.Dropdown(
    options=support_devices + ["AUTO"],
    value="CPU",
    description="Device:",
    disabled=False,
)

device

OpenVINO models can be run locally through the `HuggingFacePipeline` class in LangChain. To deploy a model with OpenVINO, you can specify the `backend="openvino"` parameter to trigger OpenVINO as backend inference framework. For [more information](https://python.langchain.com/docs/integrations/llms/openvino/).

In [ ]:
from langchain_huggingface import HuggingFacePipeline
from transformers.generation.stopping_criteria import (
    StoppingCriteriaList,
)
from transformers.tools.agents import StopSequenceCriteria

ov_config = {"PERFORMANCE_HINT": "LATENCY", "NUM_STREAMS": "1", "CACHE_DIR": ""}
stop_tokens = ["\nObservation", "<|end|>"]

ov_llm = HuggingFacePipeline.from_model_id(
    model_id=model_path,
    task="text-generation",
    backend="openvino",
    model_kwargs={
        "device": device.value,
        "ov_config": ov_config,
        "trust_remote_code": True,
    },
    pipeline_kwargs={"max_new_tokens": 4096},
)
ov_llm = ov_llm.bind(skip_prompt=True, stop=["\nObservation"])

tokenizer = ov_llm.pipeline.tokenizer
ov_llm.pipeline._forward_params["stopping_criteria"] = StoppingCriteriaList(
    [StopSequenceCriteria(stop_tokens, tokenizer)]
)

You can get additional inference speed improvement with [Dynamic Quantization of activations and KV-cache quantization] on CPU(https://docs.openvino.ai/2024/learn-openvino/llm_inference_guide/llm-inference-hf.html#enabling-openvino-runtime-optimizations). These options can be enabled with `ov_config` as follows:

In [ ]:
ov_config = {
    "KV_CACHE_PRECISION": "u8",
    "DYNAMIC_QUANTIZATION_GROUP_SIZE": "32",
    "PERFORMANCE_HINT": "LATENCY",
    "NUM_STREAMS": "1",
    "CACHE_DIR": "",
}

## Create agent

[back to top ⬆️](#Table-of-contents:)

Now that we have defined the tools, prompt template and LLM, we can create the agent_executor.

The agent executor is the runtime for an agent. This is what actually calls the agent, executes the actions it chooses, passes the action outputs back to the agent, and repeats.


In [ ]:
from langchain.agents import AgentExecutor, StructuredChatAgent

agent = StructuredChatAgent.from_llm_and_tools(
    ov_llm,
    tools,
    prefix=PREFIX,
    suffix=SUFFIX,
    human_message_template=HUMAN_MESSAGE_TEMPLATE,
    format_instructions=FORMAT_INSTRUCTIONS,
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

## Run the agent

[back to top ⬆️](#Table-of-contents:)

We can now run the agent with a math query. Before getting the final answer, a agent executor will also produce intermediate steps of reasoning and actions. The format of these messages will follow your prompt template.


In [ ]:
agent_executor.invoke(
    {
        "input": "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result"
    }
)

## Interactive Demo

[back to top ⬆️](#Table-of-contents:)

Let's create a interactive agent using [Gradio](https://www.gradio.app/).


### Use built-in tools

[back to top ⬆️](#Table-of-contents:)

LangChain has provided a list of all [built-in tools](https://python.langchain.com/docs/integrations/tools/). In this example, we will use `Wikipedia` python package to query key words generated by agent.


In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.pydantic_v1 import BaseModel


api_wrapper = WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=1000)

class WikiInputs(BaseModel):
    """Inputs to the wikipedia tool."""
    query: str


wikipedia = WikipediaQueryRun(
    description="A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.",
    args_schema=WikiInputs,
    api_wrapper=api_wrapper,
)

In [ ]:
wikipedia.invoke({"query": "OpenVINO"})

### Create customized tools

[back to top ⬆️](#Table-of-contents:)

In this examples, we will create 2 customized tools for `image generation` and `weather qurey`.

In [ ]:
import urllib.parse


@tool
def painting(prompt: str) -> str:
    """
    AI painting (image generation) service, input text description, and return the image URL drawn based on text information.
    """
    prompt = urllib.parse.quote(prompt)
    image_url = "https://image.pollinations.ai/prompt/" + prompt
    result = {"image_url": image_url}
    return str(result)


painting.invoke({"prompt": "a cat"})

In [ ]:
@tool
def weather(
    city_name: str,
) -> str:
    """
    Get the current weather for `city_name`
    """

    if not isinstance(city_name, str):
        raise TypeError("City name must be a string")

    key_selection = {
        "current_condition": [
            "temp_C",
            "FeelsLikeC",
            "humidity",
            "weatherDesc",
            "observation_time",
        ],
    }
    import requests

    resp = requests.get(f"https://wttr.in/{city_name}?format=j1")
    resp.raise_for_status()
    resp = resp.json()
    ret = {k: {_v: resp[k][0][_v] for _v in v}
           for k, v in key_selection.items()}

    return str(ret)


weather.invoke({"city_name": "London"})

### Create AI agent demo with Gradio UI

[back to top ⬆️](#Table-of-contents:)

In [ ]:
tools = [wikipedia, painting, weather]

agent = StructuredChatAgent.from_llm_and_tools(
    ov_llm,
    tools,
    prefix=PREFIX,
    suffix=SUFFIX,
    human_message_template=HUMAN_MESSAGE_TEMPLATE,
    format_instructions=FORMAT_INSTRUCTIONS,
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
import gradio as gr

examples = [
    ["Get current weather of London and draw an image for it"],
    ["What is OpenVINO ?"],
    ["Create an image of pink cat and return its URL"],
    ["How many people live in Canada ?"],
    ["What is the weather like in New York now ?"],
]


def partial_text_processor(partial_text, new_text):
    """
    helper for updating partially generated answer, used by default

    Params:
      partial_text: text buffer for storing previosly generated text
      new_text: text update for the current step
    Returns:
      updated text string

    """
    partial_text += new_text
    return partial_text


def user(message, history):
    """
    callback function for updating user messages in interface on submit button click

    Params:
      message: current message
      history: conversation history
    Returns:
      None
    """
    # Append the user's message to the conversation history
    return "", history + [[message, ""]]


def bot(history):
    """
    callback function for running chatbot on submit button click

    Params:
      history: conversation history

    """
    partial_text = ""

    for new_text in agent_executor.stream(
        {"input": history[-1][0]},
    ):
        if "output" in new_text.keys():
            partial_text = partial_text_processor(partial_text, new_text["output"])
            history[-1][1] = partial_text
            yield history


def request_cancel():
    ov_llm.pipeline.model.request.cancel()


with gr.Blocks(
    theme=gr.themes.Soft(),
    css=".disclaimer {font-variant-caps: all-small-caps;}",
) as demo:
    names = [tool.name for tool in tools]
    gr.Markdown(f"""<h1><center>OpenVINO Agent for {str(names)}</center></h1>""")
    chatbot = gr.Chatbot(height=500)
    with gr.Row():
        with gr.Column():
            msg = gr.Textbox(
                label="Chat Message Box",
                placeholder="Chat Message Box",
                show_label=False,
                container=False,
            )
        with gr.Column():
            with gr.Row():
                submit = gr.Button("Submit")
                stop = gr.Button("Stop")
                clear = gr.Button("Clear")
    gr.Examples(
        examples, inputs=msg, label="Click on any example and press the 'Submit' button"
    )

    submit_event = msg.submit(
        fn=user,
        inputs=[msg, chatbot],
        outputs=[msg, chatbot],
        queue=False,
    ).then(
        fn=bot,
        inputs=[
            chatbot,
        ],
        outputs=chatbot,
        queue=True,
    )
    submit_click_event = submit.click(
        fn=user,
        inputs=[msg, chatbot],
        outputs=[msg, chatbot],
        queue=False,
    ).then(
        fn=bot,
        inputs=[
            chatbot,
        ],
        outputs=chatbot,
        queue=True,
    )
    stop.click(
        fn=request_cancel,
        inputs=None,
        outputs=None,
        cancels=[submit_event, submit_click_event],
        queue=False,
    )
    clear.click(lambda: None, None, chatbot, queue=False)

# if you are launching remotely, specify server_name and server_port
#  demo.launch(server_name='your server name', server_port='server port in int')
# if you have any issue to launch on your platform, you can pass share=True to launch method:
# demo.launch(share=True)
# it creates a publicly shareable link for the interface. Read more in the docs: https://gradio.app/docs/
demo.launch()

In [ ]:
# please run this cell for stopping gradio interface
demo.close()